# Monitoramento semanal dos termos e tópicos de conversação (cliente X)

Objetivos

●	compreender como grupos digitais de diferentes posições no espectro político têm discutido questões sobre feminismos, pautas LGBTQI+, direitos sexuais, reprodutivos e gênero;  
●	compreender, em particular, como se dá o processo de contágio das táticas anti-gênero ou das narrativas progressistas de gênero entre os grupos;  
●	monitorar mensagens e táticas que possam gerar risco às ativistas e organizações que trabalham com temas de gênero.  

---

Método
  
Qual a prevalência dos temas identitários (conforme palavras-chave abaixo) nos grupos?     
a. Comparar a frequência das mensagens de política identitária com as mensagens gerais mais compartilhadas no período.  
b. Comparar o enquadramento das mensagens sobre políticas identitárias entre os clusters;
c. Avaliar se há algum nível de ameaça a ativistas, personalidades ou organizações (a partir dos nomes elencados abaixo)  

* Instalações e acesso ao Elastich para busca dos index

In [ ]:
!conda install -y -c conda-forge hdbscan pygraphviz 
!pip uninstall -y numpy numba
!pip install --no-cache bertopic 'bertopic[visualization]' sklearn plotly ipywidgets xlrd openpyxl elasticsearch==7.12.1 unicode-slugify certifi==2018.8.24 numpy numba

import nltk
nltk.download('punkt')

In [ ]:
import requests
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import getpass
import certifi
import urllib3

THREADS     = 10 # memórias
ES_HOST     = #sigiloso
ES_USER     = 'amanda.vasconcellos'
ES_PASS     = getpass.getpass()
ES_INDEX    = # dados
es = Elasticsearch(
    [ES_HOST],
    http_auth=(ES_USER, ES_PASS),
    timeout=60, 
    max_retries=10, 
    maxsize=THREADS, 
    retry_on_timeout=True, 
    ca_certs=certifi.where(), 
    verify_certs=False,
    ssl_show_warn=False
)

In [ ]:
from datetime import datetime, timedelta

start_date = f"{(datetime.now() - timedelta(days=6)):%Y-%m-%dT%H:%M:%S.000Z}" 
end_date = f'{datetime.now():%Y-%m-%dT%H:%M:%S.000Z}' 

messages = [] 
query = {
    "query":{
        "bool" : {
            "must" : [
                {
                    "range": {
                        "timestamp": {
                            "gte": start_date, # greater than or equal
                            "lte": end_date    # less than or equal
                        }
                    }
                },
                # {
                #     "term": {
                #         "type.keyword": {
                #             "value": "extendedText", # apenas mensagens de texto
                #             "boost": 1.0
                #         }
                #     }
                # }
            ]
        }
    }
}

try:
    data = es.search(index=ES_INDEX, body=query, scroll='2m', size=1000) 
    print("%s Got %d Hits" % (ES_INDEX,data['hits']['total']['value']))
    scroll_size = len(data['hits']['hits'])
    sid = data['_scroll_id']
    while scroll_size > 0:
        [messages.append(doc['_source']) for doc in data['hits']['hits']] 
        data = es.scroll(scroll_id=sid, scroll='2m')
        sid = data['_scroll_id']
        scroll_size = len(data['hits']['hits'])
except Exception as e:
    print(e)
    pass

* Data cleaning:

Limpeza das stopwords, tokenize e caracteres para criação de coluna "clean_content"

In [ ]:
import pandas as pd
import re
import nltk #natural language
from nltk.corpus import stopwords # palavras não significantes
from nltk.tokenize import word_tokenize # função que divide frase em palavras

nltk.download('stopwords')
nltk.download('punkt') # cria listas de palavras, para palavras abreviadas, início de frase

#stop_words = stopwords.words('english')
stop_words = nltk.corpus.stopwords.words('portuguese')
def clean_text(x):
    x = str(x)
    x = x.lower()
    x = re.sub(r'#[A-Za-z0-9]*', ' ', x)
    x = re.sub(r'https*://.*', ' ', x)
    x = re.sub(r'@[A-Za-z0-9]+', ' ', x)
    tokens = word_tokenize(x)
    x = ' '.join([w for w in tokens if not w.lower() in stop_words])
    x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-;:<=>?@[\\]^_`{|}~“…”’'), ' ', x)
    x = re.sub(r'\d+', ' ', x)
    x = re.sub(r'\n+', ' ', x)
    x = re.sub(r'\s{2,}', ' ', x)
    return x.strip().split(' ')

# msgs = list(filter(lambda x: x['message_type']=='chat',messages))
df = pd.DataFrame(messages)
df['clean_content'] = df.full_content.apply(clean_text)
df.sample(10)

* Dicionário de termos para monitoramento:

Definição de bad_words, termos de gênero, nomes de mulheres da política institucional, ativismo, ciência, jornalismo e organizações; 

As palavras chave foram selecionadas a partir de estudos prévios.

In [ ]:
# Definir objeto filtro para termos de busca

term_filters = {
    "bad_words":{
        "terms": ["filho puta", "filha puta", "filho da puta","filha da puta","filho de uma puta","puta merda","puta pariu","puta que pariu","puta queo pariu","fela da puta","fdp ","cuzão","cuzao","merda","canalha"],
        "title": "Palavrões"
    },
    "gender_identity" : {
        "terms": ["Aborto", "Abortista", "Aborteira", "Assassinato de bebês", "Bebê não nascido", "Criança não nascida", "Criança por nascer", "Com meus filhos não se meta", "Com meus filhos não te metas", "Cultura da Morte", "Depravadas", "Depravada", "Depravado", "Depravados", "Defesa da vida", "Defesa da família", "Discurso de ódio", "Demoníaca", "Doutrinação", "Doutrina nas escolas", "Educação Sexual", "Família tradicional", "Família natural", "Feminismo", "Feminista", "Feminazi", "Gay", "Gaysista", "Gayzista", "Homossexual", "Homossexualismo", "Homossexualidade", "Homem biológico", "Homem de bem", "Homens de bem", "Ideologia de gênero", "Infanticídio", "Lésbica", "Liberdade de expressão", "Liberdade Religiosa", "Machista", "Machismo", "Mulher biológica", "Mulher de bem", "Mulheres de bem", "Maníaco Sexual", "Mulher indígena", "Mulher negra", "Mulher preta", "Maconheira", "Nascituro", "Perverso", "Perversidade", "Patrulhamento ideológico", "Pró-vida", "Pro vida", "Pedofilia", "Pedófilo", "Pedófila", "Politicamente correto", "Piranha", " Puta "," puta", "Sapatão", "Sexualidade", "Satanista", "Teleaborto", "Tele aborto", "Tele-aborto", "Transexual", "Transgênero", "Traveco", "Travesti", "Vagabunda", "Vida desde a concepção", "Valor da mulher"],
        "title": "Identidade de Genêro"
    },
    "institucional_policy" : {
        "terms":["Damares Alves","Damares","Carla Zambelli","Zambelli","Talíria Petrone","Áurea Carolina","Marielle Franco","Marielle","Gleisi Hoffman","Dilma Rousseff","Dilma","Joyce Hasselman","Tabata Amaral","Erika Hilton","Erica Malunguinho","Natália Bonavides","Samia Bonfim","Samia","Monica Benício","Monica Francisco","Isa Penna","Luciana Boiteux","Luiza Erundina","Erundina","Benny Briolly","Carolina Iara","Manuela D’ávila","Erika Kokay","Vivi Reis","Jandira Feghali","Benedita da Silva","Fernanda Melchionna","Simone Tebet","Liana Cirne","Tainá de Paula","Joenia Wapichana","Marília Arraes","Renata Souza","Marina Silva","Maria do Rosário","Rosário","Rosario","Luciana Genro"],
        "title": "Política Institucional"
    },
    "activism": {
        "terms": ["Sonia Correa","Sabrina Fernandes","Sonia Guajajara","Guajajara","Alice Pataxó","Anielle Franco","Luyara Franco","Winnie Bueno","Sandra Lia Bazzo","Camila Mantovani","Simony dos Anjos","Txai Suruí","Vilma Reis","Sueli Carneiro","Andreza Delgado","Jurema Werneck","Lola Aronovich","Marilia Moscou","Lana de Holanda","Amara Moira","Buba Aguiar","Debora Baldin","Carina Vitral","Brenda Safra","Angela Freitas","Paula Guimarães","Paula Viana","Laura Molinari","Leina Peres","Morgani Guzzo"," Natalia Veras","Mariana Prandini","Gabriela Rondon","Luciana Brito","Rebeca Mendes","Fernanda Vicari","Luciana Viegas","Joice Berth","Jacira Melo","Lúcia Xavier","Silvia Camurça","Ligia Cardieri" ],
        "title": "Ativistas"
    },
    "science":{
        "terms": ["Debora Diniz","Marcia Tiburi","Emanuelle Goes","Flávia Biroli","Rosana Pinheiro-Machado","Isabela Kalil","Helena Paro","Carla Akotirene","Djamila Ribeiro","Monica de Bolle","Gabriela Priolli","Laura Carvalho","Natalia Pasternak","Nina da Hora","Ethel Maciel","Juliana Borges","Melania Amorim"],
        "title": "Cientistas"
    },
    "journalism": {
        "terms": ["Amanda Audi","Flávia Oliveira","Giulliana Bianconi","Helena Bertho","Fabiana Moraes","Maju Coutinho"," Bárbara Libório","Thais Folego","Lola Ferreira","Juliana Dal Piva","Mariana Varella","Eliana Alves Cruz","Andrea Martinelli","Marie Declerq","Andrea Dip","Nayara Felizardo","Miriam Leitão","Vera Magalhães","Cecília Olliveira","Patrícia Campos Melo","Mara Régia" ],
        "title" : "Jornalistas"
    },
    "organizations": {
        "terms": [" Anis "," Anis.","Grupo Curumim","Nem Presa Nem Morta","Portal Catarinas"," GHS "," GHS.","Coletivo Margarida Alves", "Margarida Alves", "Rede Feminista de Saúde", " Criola","Geledes","Coletivo Feminista de Sexualidade e Saúde"," Cepia","Católicas pelo direito de decidir","Frente Nacional Pela Legalização do Aborto","FNPLA","Cunhã Feminista","CFEMEA","SOS Corpo","REDEH"," CLADEM","Redes da Maré"," AMB "," AMB.","Articulação de Mulheres Brasileiras"," Nossas ","Bloco A"," SPW ","Coalizão Negra por Direitos","Think Olga","Instituto Patrícia Galvão","Gênero e Número","Midia Ninja","Mídia Ninja"],
        "title":"Organizações"
    }
}

In [ ]:
def find_terms(x): # gerar coluna de identificação de termos e palavra chave pelo 'clean_content'
    x = str(x)

    return_keys = [] #criar lista
    for key in term_filters.keys():
        for value in term_filters[key]['terms']: 
            if x.lower().find(value.lower()) > -1: # se os valores forem encontrados
                return_keys.append( (key,value) )
    return return_keys if len(return_keys)>0 else None

df['_keys_'] = df.full_content.apply(find_terms) # coluna de lista de tupla [(termo, palavra_chave)]
df._keys_.value_counts()

# output: [(gender_identity, Gay)]

In [ ]:
def explode_keys(row): # gerar colunas únicas de termos
    columns = []
    if row['_keys_']:
        for t in row['_keys_']:
            if t[0] in row.keys():
                row[t[0]].append(t[1])
            else: 
                row[t[0]] = [t[1]]
    return row
    
        
df2 = df.apply(explode_keys,axis=1)
df2.columns

# output: '_keys_', 'bad_words', 'gender_identity'

* Visualização dos termos por tempo

In [ ]:
#incidência dos termos de busca por tempo e por flag

df2['keys_bool'] = df2._keys_.apply(lambda x: True if type(x) is list else False) # Se houver lista de tupla retornar True
df2[['timestamp','keys_bool']].groupby([df2["timestamp"].astype("datetime64").dt.date,'keys_bool'])\ # agrupar flags por tempo
    .count()\
    .rename(columns={'timestamp':'count'})\
    .reset_index()\
    .fillna(0)\
    .sort_values(by='timestamp', ascending=True)\
    .pivot(index='timestamp',columns='keys_bool',values='count')\
    .plot(kind="bar",template='plotly_dark',height=600)

# output: histograma da frequência de aparecimento dos termos monitorados no tempo

In [ ]:
# frequência de aparecimento dos termos

pd.options.plotting.backend = "plotly"

# remove primeiro valor que é o array vazio
temp=df3._keys_.value_counts()[0:30] # top 30 termos
# temp.index[0][0][0] # 'gender_identity'

# constroi um dicionário somando a frequencia dos termos de forma individual
graph = {}
for i,v in enumerate(temp):
    for k in temp.index[i]:
        key = f'{k[0]}__{k[1]}'
        if graph.get(key):
            graph[key] += v
        else:
            graph[key] = v

#transforma o dicionário em tuplas e as tuplas em um DataFrame
df01_3 = pd.DataFrame(graph.items(),columns=['termo','frequencia']) 

df01_3.sort_values('frequencia',ascending=False).plot(x='termo',y='frequencia',kind='bar',template='plotly_dark',height=600).show()

# output: plota o grafico de barras de termos encontrados

In [ ]:
# plotar a frequeência dos termos e palavras chave separadamente ao longo do tempo

df2['identidade'] = df2[pd.isna(df2.bad_words)].gender_identity.fillna('sem_ocorrencia').astype(str) # identidade = termos de genero sem as bad_words
df2['identidade'] = df2.identidade.apply(lambda x: str(x)[:30]) 
df2[df2['identidade']!='sem_ocorrencia'][['timestamp','identidade']]\
    .groupby(
        [ 
            df2["timestamp"].astype("datetime64").dt.date,
            'identidade'
        ]
    )\
    .count()\
    .rename(columns={'timestamp':'count'})\
    .reset_index()\
    .sort_values(by='timestamp', ascending=True)\
    .pivot(index='timestamp',columns='identidade',values='count')\
    .plot(kind="bar",template='plotly_dark',height=600)

# output: gráfico de barras empilhadas com as principais palavras chave por dia (frequência absoluta)

In [ ]:
# métrica percentual

df2['identidade_bool'] = df2.gender_identity.apply(lambda x: True if type(x) is list else False)
df01 = df2[pd.isna(df2.bad_words)][['timestamp','identidade_bool']]\
    .groupby([df2["timestamp"].astype("datetime64").dt.date,'identidade_bool'])\
    .count()\
    .rename(columns={'timestamp':'count'})\
    .reset_index()\
    .fillna(0)\
    .sort_values(by='timestamp', ascending=True)\
    .pivot(index='timestamp',columns='identidade_bool',values='count')

df01.columns = ['False','True']
df01['total'] = df01[['False','True']].fillna(0).apply(lambda row: row['True'] + row['False'],axis=1)

df01['sem_ocorrencia'] = df01.apply(lambda row: row['False']/row['total']*100,axis=1)
df01['com_ocorrencia'] = df01.apply(lambda row: row['True']/row['total']*100,axis=1)

df01[['sem_ocorrencia','com_ocorrencia']].plot(kind="bar",template='plotly_dark',height=400)

# output: gráfico de barras com colunas percentuais das frequências identificadas por flags

* Gerando dataset para a equipe de análise de dados

Objetivo: fornecer para a equipe alguns exemplos de ocorrências que se adequam ao modelo metodológico

In [ ]:
#criando filtro para identificação de mensagens com termos de gênero:

filtro_gender = df2.loc[(df2.identidade_bool != False)] # onde a flag para gênero é True

filtro_gender.head(5)

In [ ]:
ocorrencias_gender = filtro_gender.groupby([filtro_gender.content, filtro_gender.identidade])\ # agregando conteúdo da mensagem True
[['content','group']]\
    .agg({
        'content':['count'], # gerando frequencia de aparecimento da mesma mensagem
         'group':['count'] # frequência de repetição
        
    })\
    .reset_index()

ocorrencias_gender.columns = ['content', 'gender_key','content_count', 'group_count']
ocorrencias_gender.sort_values(by='content_count', ascending=False)

# output: dataset organizado em 4 colunas 
#(conteúdo integral da mensagem, chave de gênero identificada, frequência absoluta de aparecimento da mensagem e frequência de grupos enviados)

In [ ]:
# exportar csv 

ocorrencias_gender.to_csv('ocorrencias_gender_agrupadas.csv', index=False)

#obs: podem ser criados filtros para qualquer tipo de termo monitorado

* Identificação de tópicos de conversação

Utilizando o BERTopic (modelagem de tópicos por classe)

In [ ]:
#modelagem de tópicos por classe

from bertopic import BERTopic #modelagem
import os #sistema operacional

os.environ["TOKENIZERS_PARALLELISM"] = 'true'

topic_model = BERTopic(language="portuguese", calculate_probabilities=True, verbose=True, low_memory=False) #calculando probabilidade

chat_topics = []
docs = []
timestamps = []

for chat_id in df.chat_id.value_counts()[df.chat_id.value_counts() > 10].index.values: # para grupos com mais de 10 partifcipantes na conversa
    [docs.append(doc) for doc in df[df.chat_id == chat_id]['clean_content'].values] # utilizando a mesma coluna de mensagens 'clean_content'
    [timestamps.append(doc) for doc in df[df.chat_id == chat_id]['timestamp'].values] # append timestamp

topics, probs = topic_model.fit_transform(docs) # tranformandoo modelo pelos dados (docs)
freq = topic_model.get_topic_info() # Observar as frequências dos tópicos mais comuns e dos menos frequentes (-1) que serão provavlemente ignorados

# output: dataset contendo os tópicos agrupados por BERTopic e as respectivas frequências

In [ ]:
# Após treinar o modelo, gerar as visualizações para melhor entendimento dos tópicos gerados

topic_model.visualize_topics()

#output: gráfico de visualização dos tópicos, seus tamanho ao longo dos eixos e proximidade semântica

In [ ]:
# Aqui é feita a hierarquização dos tópicos (n foi definido através e clusteriação prévia)

topic_model.visualize_hierarchy(top_n_topics=20)

#output: relação hierárquica entre os tópicos de conversação

In [ ]:
# Visualizar termos por tópicos

topic_model.visualize_barchart(top_n_topics=10)

#output: Gráficos de barra com as frequências de termos por tópicos (Aqui comparamos os "term_filters" gerados na primeira parte

In [ ]:
# matriz de similaridade

topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

# output: semelhança de cosseno pelos embeddings (avaliar possíveis tópicos correlacionados e o repercussão dos tópicos)

* Clusterizando os tipos de senders (usuários) pelo conteúdo das mensagens (content)

Utilizando Kmeans

In [ ]:
# Clustering senders by content
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt
%matplotlib inline

import nltk

from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = nltk.corpus.stopwords.words('portuguese') # data cleaning

vectorizer = TfidfVectorizer(stop_words=stop_words)
df['content_len'] = df.content.astype(str).apply(lambda x: len(x.strip())) # valores de content
cdf = df[(~pd.isna(df['content'])) & (df['content_len']>1) ].drop_duplicates(subset=['content'],keep='first') # documentos 

documents = cdf.clean_content.apply(lambda x: ' '.join(x) if x[0] != None else x) 
# documents
X = vectorizer.fit_transform(documents) 

cost = []
K = range(1,25)
for num_clusters in list(K):
    print(f'Calculating with {num_clusters}',end='\r')
    km = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=1000, n_init=5, verbose=0)
    km.fit_predict(X)
    cost.append(km.inertia_)
    
plt.plot(K, cost, 'bx-')
plt.xlabel('No. of clusters')
plt.ylabel('Cost')
plt.title('Elbow Method For Optimal k')
plt.show()

# output: gráfico dos valores de K (melohores números de clusters pelos tópicos), Ex: 23

In [ ]:
true_k = 23 # valor de k 
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=1000, n_init=1)
model.fit(X) 

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(true_k):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])
        
# output: top terms por cluster. 
# Ex: cluster 1: mulheres, luta, brasil, deus

In [ ]:
# transfer predictions to dataset

def content_clustering(x):
    Y = vectorizer.transform(x)
    prediction = model.predict(Y)
    return prediction[0]

ccs = df[(~pd.isna(df['content'])) & (df['content_len']>1)].drop_duplicates(subset=['content'],keep='first')[['content','clean_content']].values
for i,item in enumerate(ccs):
    print(f'{i}/{len(ccs)}',end='\r')
    filtro = (df02.content == item[0])
    df02.content_cluster.value_counts(dropna=False)
    
# output: dataset original com mensagens clusterizadas por tópicos em "content_cluster"
# Ex: "content": Mulheres no Brasil representam o maior número de trabalhadores informais; "content_cluster": cluster 1